# MAP-Elites track generations

In [ ]:
import numpy as np
import requests
import random
import matplotlib.pyplot as plt
import glob, os
import joblib    
# Just load a saved t-SNE or UMAP model     
EMBEDDING_MODEL = joblib.load("embedding/umap_model.joblib")   # 2-D model trained offline


from ribs.archives import SlidingBoundariesArchive
from ribs.emitters import EmitterBase
from ribs.schedulers import Scheduler
from dask.distributed import Client, LocalCluster, as_completed
from ribs.visualize import sliding_boundaries_archive_heatmap


In [ ]:
BASE_URL = 'http://localhost:4242'
GENERATION_MODE = 'voronoi' #'convexHull' or 'voronoi'
POINTS_COUNT = 100
MAX_SELECTED_CELLS = 10
SOLUTION_DIM = POINTS_COUNT * 2 + MAX_SELECTED_CELLS * 2 + 1 
TRACK_SIZE_RANGE = (4, 10)
LENGTH_RANGE = (400, 2000)
ITERATIONS = 500
ARCHIVE_DIM = 3
INIT_POPULATION = ARCHIVE_DIM * ARCHIVE_DIM 

INVALID_SCORE    = -1e9
CHECKPOINT_EVERY = 50

DEBUG_CROSSOVER = True
DEBUG_MUTATION = True


ARCHIVE_BINS = 30                               # cells per axis
REMAPPING_EVERY = 200                           # move boundaries every 200 insertions
BUFFER_SIZE = 1000                              # keep last 1000 solutions

In [ ]:
cluster = LocalCluster(processes=True, n_workers=5, threads_per_worker=1)
client = Client(cluster)

### Helper functions

In [ ]:
def generate_solution(iteration):
    print(f"Generating solution for iteration {iteration}")
    try:
        response = requests.post(
            f"{BASE_URL}/generate",
            json={
                "id": iteration + random.random(),
                "mode": GENERATION_MODE,
                "trackSize": random.randint(TRACK_SIZE_RANGE[0], TRACK_SIZE_RANGE[1])
            },
            timeout=60
        )
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error generating solution for iteration {iteration}: {e}")
        return None

def solution_to_array(sol):
    if sol is None:
        return None
    arr = np.zeros(SOLUTION_DIM)
    for i, p in enumerate(sol.get("dataSet", [])):
        arr[i * 2] = p.get("x", 0)
        arr[i * 2 + 1] = p.get("y", 0)
    for i, c in enumerate(sol.get("selectedCells", [])):
        if i < MAX_SELECTED_CELLS:
            idx = POINTS_COUNT * 2 + i * 2
            arr[idx] = c.get("x", 0)
            arr[idx + 1] = c.get("y", 0)
    arr[-1] = sol.get("id", 0)
    return arr

def array_to_solution(arr):
    ds = []
    for i in range(0, POINTS_COUNT * 2, 2):
        ds.append({"x": float(arr[i]), "y": float(arr[i+1])})
    sel = []
    for i in range(POINTS_COUNT * 2, SOLUTION_DIM - 1, 2):
        x_val = arr[i]
        y_val = arr[i+1]
        if x_val != 0 or y_val != 0:
            sel.append({"x": float(x_val), "y": float(y_val)})
    return {
        "id": float(arr[-1]),
        "mode": GENERATION_MODE,
        "dataSet": ds,
        "selectedCells": sel
    }

def get_fractional_part(x):
    return x - int(x)

def pca_align(points):
    pts = points - points.mean(0)
    u, _, _ = np.linalg.svd(pts, full_matrices=False)
    angle = np.arctan2(u[1, 0], u[0, 0])
    rot = np.array([[np.cos(-angle), -np.sin(-angle)],
                    [np.sin(-angle),  np.cos(-angle)]])
    aligned = pts @ rot.T
    if aligned[0, 0] < 0:
        aligned[:, 0] *= -1
    return aligned

def descriptor_from_track(sol):
    pts = np.array([[p["x"], p["y"]] for p in sol.get("splineVector", [])], dtype=float)
    aligned = pca_align(pts)
    flat = aligned.ravel()
    return EMBEDDING_MODEL.transform(flat[None, :])[0]

def fitness_formula(fit):
    s  = fit["speed_entropy"]
    ov = fit["total_overtakes"]
    dx = abs(fit["deltaX"])          # use magnitude only
    dx = max(dx, 1e-3)               # avoid divide-by-zero blow-ups
    return s + ov / dx

def evaluate_solution(sol):
    sol_id = sol.get("id", 0)
    ok = True
    msg = ""
    score = -9999
    try:
        r = requests.post(f"{BASE_URL}/evaluate", json=sol, timeout=60)
        r.raise_for_status()
        r_json = r.json()
        fit = r_json.get("fitness", {})
        score = fitness_formula(fit)
        desc = descriptor_from_track(r_json)  
    except Exception as e:
        ok = False
        msg = str(e)
        desc = np.zeros((2,))  # just in case of error

    return sol_id, ok, msg, score, desc



## Genetic operators

In [ ]:
class CustomEmitter(EmitterBase):
    def __init__(self, archive, solution_dim, batch_size=ARCHIVE_DIM, bounds=None):
        super().__init__(archive, solution_dim=solution_dim, bounds=bounds)
        self.batch_size = batch_size
        self.iteration = 0

    def ask(self):
        self.iteration += 1
        print(f"Emitter.ask() called for iteration {self.iteration}")
        if self.iteration <= INIT_POPULATION:
            out = []
            for _ in range(self.batch_size):
                sol = generate_solution(self.iteration - 1)
                arr = solution_to_array(sol)
                if arr is not None:
                    out.append(arr)
                else:
                    out.append(np.full(SOLUTION_DIM, -9999))
            return np.array(out)
        else:
            if random.random() < 0.5:
                return self.mutate_solutions()
            else:
                return self.crossover_solutions()

    def mutate_solutions(self):
        print(f"Mutating solutions for iteration {self.iteration}")
        parents = self.archive.sample_elites(self.batch_size)
        out = []
        for i in range(self.batch_size):
            arr = parents["solution"][i]
            sol = array_to_solution(arr)
            try:
                response = requests.post(
                    f"{BASE_URL}/mutate",
                    json={
                        "individual": sol,
                        "intensityMutation": 10
                    },
                    timeout=60
                )
                response.raise_for_status()
                mutated = response.json().get("mutated", {})
                frac = get_fractional_part(sol["id"])
                mutated["id"] = self.iteration - 1 + frac
                mutated_arr = solution_to_array(mutated)
                if mutated_arr is not None:
                    out.append(mutated_arr)
                    print(f"Mutated ID={sol['id']} to ID={mutated['id']}")
                else:
                    out.append(np.full(SOLUTION_DIM, -9999))
            except requests.RequestException as e:
                print(f"Error mutating solution ID={sol['id']}: {e}")
                out.append(np.full(SOLUTION_DIM, -9999))
        return np.array(out)

    def crossover_solutions(self):
        print(f"Crossover solutions for iteration {self.iteration}")
        out = []
        for _ in range(self.batch_size // 2):
            try:
                while True:
                    parents = self.archive.sample_elites(2)
                    sol1 = array_to_solution(parents["solution"][0])
                    sol2 = array_to_solution(parents["solution"][1])
                    if sol1["id"] != sol2["id"]:
                        break
                response = requests.post(
                    f"{BASE_URL}/crossover",
                    json={
                        "mode": GENERATION_MODE,
                        "parent1": sol1,
                        "parent2": sol2
                    },
                    timeout=60
                )
                response.raise_for_status()
                offspring = response.json().get("offspring", {})
                f1 = get_fractional_part(sol1["id"])
                f2 = get_fractional_part(sol2["id"])
                frac = (f1 + f2) % 1
                child_id = self.iteration - 1 + frac
                child_sol = {
                    "id": child_id,
                    "mode": GENERATION_MODE,
                    "trackSize": len(offspring.get("sel", [])),
                    "dataSet": offspring.get("ds", []),
                    "selectedCells": offspring.get("sel", [])
                }
                child_arr = solution_to_array(child_sol)
                if child_arr is not None:
                    out.append(child_arr)
                    print(f"Crossover Parent1 ID={sol1['id']}, Parent2 ID={sol2['id']} => Child ID={child_id}")
                else:
                    out.append(np.full(SOLUTION_DIM, -9999))
            except requests.RequestException as e:
                print(f"Error during crossover: {e}")
                out.append(np.full(SOLUTION_DIM, -9999))
        return np.array(out)

## Illuminating search spaces by mapping elites


In [ ]:
import glob
import os

archive = SlidingBoundariesArchive(
    solution_dim=SOLUTION_DIM,
    dims=[ARCHIVE_BINS, ARCHIVE_BINS],          # 2‑D descriptor space
    ranges=[(-1, 1), (-1, 1)],                  # initial bounds in each dim
    remap_frequency=REMAPPING_EVERY,
    buffer_capacity=BUFFER_SIZE
)

emitter = CustomEmitter(
    archive,
    solution_dim=SOLUTION_DIM,
    batch_size=INIT_POPULATION,
    bounds=[(0, 600)] * (SOLUTION_DIM - 1) + [(0, float('inf'))]
)

scheduler = Scheduler(archive, [emitter])

# ──────────────────────────────────────────────────────────────
# Resume from latest checkpoint (if any)
# ──────────────────────────────────────────────────────────────
checkpoint_files = sorted(glob.glob("checkpoint_*.npz"))
start_iter = 0
if checkpoint_files:
    latest = checkpoint_files[-1]
    archive.load(latest)
    start_iter = int(os.path.splitext(latest)[0].split("_")[1])
    emitter.iteration = start_iter
    print(f"[Resume] Loaded {latest}, continuing from iteration {start_iter+1}")
else:
    print("[Resume] No checkpoint found – starting fresh.")


def run_map_elites(total_iters, start_iter=0):
    global_best_score = INVALID_SCORE
    global_best_id    = None
    for i in range(start_iter, total_iters):
        print(f"=== Starting iteration {i+1} ===")
        try:
            sols      = scheduler.ask()
            sol_dicts = [array_to_solution(s) for s in sols]
            futs      = [client.submit(evaluate_solution, sol) for sol in sol_dicts]
            gathered  = [f.result() for f in as_completed(futs)]

            obj_list = []
            clean    = []
            for sol_id, ok, msg, score, desc in gathered:
                if not ok or not np.isfinite(score):
                    print(f"Warning: invalid score for solution ID={sol_id}, reason: {msg}")
                    score = INVALID_SCORE
                else:
                    print(f"Solution ID={sol_id} evaluated with score={score:.2f}")
                    if score > global_best_score:
                        global_best_score = score
                        global_best_id    = sol_id
                clean.append((score, desc))
                obj_list.append(score)

            obj_batch, meas_batch = zip(*clean)
            scheduler.tell(list(obj_batch), list(meas_batch))

            batch_best = max(obj_list) if obj_list else INVALID_SCORE
            print(f"Iteration {i+1} ended. Best in batch = {batch_best:.2f}")
            if global_best_id is not None:
                print(f"Global Best Score so far: {global_best_score:.2f} (ID={global_best_id})")

            data = archive.data()
            if len(data) > 0:
                arch_obj = data["objective"]
                mean_val = np.mean(arch_obj[arch_obj != INVALID_SCORE])
                best_val = np.max(arch_obj[arch_obj != INVALID_SCORE])
                cov      = archive.stats.coverage
                print(f"Archive size={len(archive)}, Coverage={cov:.3f}, "
                      f"Mean={mean_val:.2f}, Best={best_val:.2f}")
            else:
                print("Archive empty so far")

            if (i + 1) % CHECKPOINT_EVERY == 0:
                archive.save(f"checkpoint_{i+1:04d}.npz")
                print(f"[Checkpoint] Saved checkpoint_{i+1:04d}.npz")

            # ───────────────── Plot (every 5 iterations) ─────────────────
            if (i + 1) % 5 == 0:
                arch_obj   = data["objective"]
                valid_mask = arch_obj != INVALID_SCORE
                if np.any(valid_mask):
                    vmin = arch_obj[valid_mask].min()
                    vmax = arch_obj[valid_mask].max()
                else:                 # no valid points yet
                    vmin = vmax = 0.0

                plt.figure(figsize=(8, 6))
                sliding_boundaries_archive_heatmap(
                    archive,
                    boundary_lw=0.5,
                    vmin=vmin,
                    vmax=vmax,
                )
                plt.title(f"Archive Heat-map – Iteration {i+1}")
                plt.xlabel("UMAP-1")
                plt.ylabel("UMAP-2")
                plt.tight_layout()
                plt.savefig(f"archive_heatmap_iter_{i+1}.png")
                plt.close()

        except Exception as e:
            print(f"Error in iteration {i+1}: {e}")
            raise


run_map_elites(ITERATIONS, start_iter)


In [ ]:
run_map_elites(ITERATIONS, start_iter)

## Visualize Results

In [ ]:
print("All iterations complete.")
print(f"Final archive size={len(archive)}, Coverage={archive.stats.coverage:.3f}")
plt.figure(figsize=(6,5))
grid_archive_heatmap(archive)
plt.title("Final Archive Heatmap")
plt.xlabel("Speed Entropy")
plt.ylabel("Mean Gaps")
plt.savefig("final_archive_heatmap.png")
plt.show()